## Overview

This notebook is for the PayTM coding challenge - MLE position

In [0]:
from pyspark.sql.functions import isnan, when, count, col, desc, avg, rank
from pyspark.sql.functions import to_date, row_number
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import Imputer

In [0]:
# File location and type
file_location = "/FileStore/tables/data/2019/*.gz"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

STN---,WBAN,YEARMODA,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
10260,99999,20190101,26.1,21.2,1001.9,987.5,20.6,9.0,15.9,29.7,29.8,21.7*,0.02G,18.5,1000
10260,99999,20190102,24.9,22.1,1020.1,1005.5,5.4,5.6,13.6,22.1,27.1*,20.7,0.48G,22.8,1000
10260,99999,20190103,31.7,29.1,1008.9,994.7,13.6,11.6,21.4,49.5,37.4*,26.8*,0.25G,999.9,11000
10260,99999,20190104,32.9,30.3,1011.4,997.1,15.8,4.9,7.8,10.9,36.1,31.8,0.52G,999.9,1000
10260,99999,20190105,35.5,33.0,1015.7,1001.4,12.0,10.4,13.6,21.0,38.5*,32.7,0.02G,23.6,10000
10260,99999,20190106,38.5,34.1,1008.2,994.2,12.8,10.0,17.5,28.9,41.4,33.8*,0.12G,23.2,10000
10260,99999,20190107,32.1,29.8,996.8,982.7,6.9,11.3,15.5,28.6,35.1*,30.4,0.00G,999.9,1000
10260,99999,20190108,31.6,28.0,997.4,983.3,22.9,5.9,11.7,19.0,34.3,28.0*,0.53G,0.4,11000
10260,99999,20190109,29.9,27.7,1011.6,997.3,29.8,7.6,15.2,26.6,32.4,26.1,0.20G,23.6,1000
10260,99999,20190110,33.1,30.6,979.1,965.3,5.3,17.8,24.9,41.8,41.4,28.8*,0.00G,999.9,11000


In [0]:
df.printSchema()

root
-- STN---: integer (nullable = true)
-- WBAN: integer (nullable = true)
-- YEARMODA: integer (nullable = true)
-- TEMP: double (nullable = true)
-- DEWP: double (nullable = true)
-- SLP: double (nullable = true)
-- STP: double (nullable = true)
-- VISIB: double (nullable = true)
-- WDSP: double (nullable = true)
-- MXSPD: double (nullable = true)
-- GUST: double (nullable = true)
-- MAX: string (nullable = true)
-- MIN: string (nullable = true)
-- PRCP: string (nullable = true)
-- SNDP: double (nullable = true)
-- FRSHTT: integer (nullable = true)

In [0]:
df = df.withColumnRenamed("STN---","STN_NO")

In [0]:
df.show(4)

+------+-----+--------+----+----+------+------+-----+----+-----+----+-----+-----+-----+-----+------+
STN_NO| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD|GUST| MAX| MIN| PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+------+-----+----+-----+----+-----+-----+-----+-----+------+
 10260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9|29.7| 29.8|21.7*|0.02G| 18.5| 1000|
 10260|99999|20190102|24.9|22.1|1020.1|1005.5| 5.4| 5.6| 13.6|22.1|27.1*| 20.7|0.48G| 22.8| 1000|
 10260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4|49.5|37.4*|26.8*|0.25G|999.9| 11000|
 10260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9| 7.8|10.9| 36.1| 31.8|0.52G|999.9| 1000|
+------+-----+--------+----+----+------+------+-----+----+-----+----+-----+-----+-----+-----+------+
only showing top 4 rows

In [0]:
# Create a view or table

temp_table_name = "globalweathertable"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `globalweathertable`

STN---,WBAN,YEARMODA,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
10260,99999,20190101,26.1,21.2,1001.9,987.5,20.6,9.0,15.9,29.7,29.8,21.7*,0.02G,18.5,1000
10260,99999,20190102,24.9,22.1,1020.1,1005.5,5.4,5.6,13.6,22.1,27.1*,20.7,0.48G,22.8,1000
10260,99999,20190103,31.7,29.1,1008.9,994.7,13.6,11.6,21.4,49.5,37.4*,26.8*,0.25G,999.9,11000
10260,99999,20190104,32.9,30.3,1011.4,997.1,15.8,4.9,7.8,10.9,36.1,31.8,0.52G,999.9,1000
10260,99999,20190105,35.5,33.0,1015.7,1001.4,12.0,10.4,13.6,21.0,38.5*,32.7,0.02G,23.6,10000
10260,99999,20190106,38.5,34.1,1008.2,994.2,12.8,10.0,17.5,28.9,41.4,33.8*,0.12G,23.2,10000
10260,99999,20190107,32.1,29.8,996.8,982.7,6.9,11.3,15.5,28.6,35.1*,30.4,0.00G,999.9,1000
10260,99999,20190108,31.6,28.0,997.4,983.3,22.9,5.9,11.7,19.0,34.3,28.0*,0.53G,0.4,11000
10260,99999,20190109,29.9,27.7,1011.6,997.3,29.8,7.6,15.2,26.6,32.4,26.1,0.20G,23.6,1000
10260,99999,20190110,33.1,30.6,979.1,965.3,5.3,17.8,24.9,41.8,41.4,28.8*,0.00G,999.9,11000


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

#permanent_table_name = "global_weather_table"

#df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
# File location and type
file_location = "/FileStore/tables/countrylist.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
country_list = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(country_list)

COUNTRY_ABBR,COUNTRY_FULL
AA,ARUBA
AC,ANTIGUA AND BARBUDA
AF,AFGHANISTAN
AG,ALGERIA
AI,ASCENSION ISLAND
AJ,AZERBAIJAN
AL,ALBANIA
AM,ARMENIA
AN,ANDORRA
AO,ANGOLA


In [0]:
# File location and type
file_location = "/FileStore/tables/stationlist.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
station_list = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(station_list)



STN_NO,COUNTRY_ABBR
012240,NO
020690,SW
020870,SW
021190,SW
032690,UK
033450,UK
039290,UK
039790,EI
040480,IC
041300,IC


In [0]:
country_by_station = station_list.join(country_list, how = 'left', on = ['COUNTRY_ABBR'])
country_by_station.show(5)

+------------+------+--------------+
COUNTRY_ABBR|STN_NO| COUNTRY_FULL|
+------------+------+--------------+
 NO|012240| NORWAY|
 SW|020690| SWEDEN|
 SW|020870| SWEDEN|
 SW|021190| SWEDEN|
 UK|032690|UNITED KINGDOM|
+------------+------+--------------+
only showing top 5 rows

In [0]:
#Checking for null values post left-join
country_by_station.select([count(when(isnan(c), c)).alias(c) for c in country_by_station.columns]).show()

+------------+------+------------+
COUNTRY_ABBR|STN_NO|COUNTRY_FULL|
+------------+------+------------+
 0| 0| 0|
+------------+------+------------+

In [0]:
global_weather_table = df.join(country_by_station, how = 'left', on = 'STN_NO')
global_weather_table.show()

+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+-------------+
STN_NO| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|COUNTRY_ABBR| COUNTRY_FULL|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+-------------+
725390|14836|20190101|32.0|30.4|1017.6|982.9| 7.5| 7.4| 13.0| 20.0| 39.0| 27.0|0.80G|999.9| 11000| US|UNITED STATES|
725390|14836|20190102|27.1|26.5|1024.6|990.5| 3.3| 5.2| 9.9|999.9| 34.0| 24.1|0.02G|999.9| 1000| US|UNITED STATES|
725390|14836|20190103|31.1|25.3|1015.6|983.0| 9.2|12.2| 17.1| 22.0| 37.9| 24.1|0.05G|999.9| 1000| US|UNITED STATES|
725390|14836|20190104|36.6|28.3|1009.8|977.6| 10.0| 9.5| 15.0| 21.0| 50.0| 28.9|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190105|34.8|30.0|1007.6|975.3| 7.2| 4.5| 14.0|999.9| 50.0| 24.1|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190106|36.2|29.1|1021.2|988.3| 10.0| 6.1| 12.0|999.9| 50.0| 24.1|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190107|34.3|27.4|1020.0|985.3| 8.9|15.5| 22.9| 31.1| 48.9| 28.0|0.01G|999.9| 11000| US|UNITED STATES|
725390|14836|20190108|43.0|38.9|1006.5|974.3| 8.2|11.9| 26.0| 35.0| 51.1| 30.9|0.20G|999.9| 10000| US|UNITED STATES|
725390|14836|20190109|26.8|19.0|1016.6|983.1| 8.9|14.0| 18.1| 28.9| 44.1| 21.0|0.01G|999.9|101000| US|UNITED STATES|
725390|14836|20190110|22.4|14.4|1025.5|992.5| 9.6|11.2| 15.9| 26.0| 30.9| 19.0|0.01G|999.9| 1000| US|UNITED STATES|
725390|14836|20190111|21.5|14.1|1029.9|996.6| 10.0| 3.0| 7.0|999.9| 27.0| 16.0|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190112|27.7|19.3|1030.6|997.4| 10.0| 6.4| 11.1|999.9|30.2*|26.1*|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190113|24.4|16.1|1030.5|997.2| 10.0| 9.5| 15.0| 21.0| 30.9| 19.0|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190114|21.1|15.9|1028.0|994.9| 10.0| 3.3| 8.9|999.9| 34.0| 12.0|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190115|27.2|24.7|1022.1|989.3| 4.8| 9.8| 15.0| 22.0| 34.0| 12.0|0.00G|999.9| 0| US|UNITED STATES|
725390|14836|20190116|27.5|22.7|1019.8|987.4| 8.8|10.1| 17.1| 22.0| 32.0| 18.0|0.01G|999.9| 11000| US|UNITED STATES|
725390|14836|20190117|22.7|17.4|1024.9|990.6| 9.1| 5.2| 11.1| 15.0| 32.0| 15.1|0.01A|999.9| 1000| US|UNITED STATES|
725390|14836|20190118|27.9|24.0|1020.1|986.6| 5.8| 5.0| 11.1|999.9| 30.9| 16.0|0.01A|999.9| 1000| US|UNITED STATES|
725390|14836|20190119|19.6|15.2|1021.1|988.8| 3.8|10.0| 17.1| 25.1| 30.0| 15.1|0.17G|999.9|101000| US|UNITED STATES|
725390|14836|20190120| 5.4|-2.7|1022.1|988.0| 9.6|10.3| 15.0| 22.9| 21.9| -2.0|0.13G| 2.0| 1000| US|UNITED STATES|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+-------------+
only showing top 20 rows

In [0]:
#Checking for null values in gloabl weather table
global_weather_table.select([count(when(isnan(c), c)).alias(c) for c in global_weather_table.columns]).show()

+------+----+--------+----+----+---+---+-----+----+-----+----+---+---+----+----+------+------------+------------+
STN_NO|WBAN|YEARMODA|TEMP|DEWP|SLP|STP|VISIB|WDSP|MXSPD|GUST|MAX|MIN|PRCP|SNDP|FRSHTT|COUNTRY_ABBR|COUNTRY_FULL|
+------+----+--------+----+----+---+---+-----+----+-----+----+---+---+----+----+------+------------+------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+----+--------+----+----+---+---+-----+----+-----+----+---+---+----+----+------+------------+------------+

In [0]:
input_col = []
imputer = Imputer(
    inputCols=df.columns, 
    outputCols=["{}_imputed".format(c) for c in df.columns]
)
imputer.fit(df).transform(df)

imr = Imputer(missing_values='NaN', strategy='median', axis=0)
imr = imr.fit(data[['age']])

In [0]:
global_weather_table = global_weather_table.withColumn("TEMP", \
              when(global_weather_table["TEMP"] == 9999.9, None).otherwise(global_weather_table["TEMP"]))

In [0]:
#could not finish the imputation
imputer = Imputer(
    inputCols=global_weather_table['TEMP'], 
    outputCols=["{}_imputed".format(c) for c in df['TEMP']]
    ).setStrategy("median")

# Add imputation cols to df
df = imputer.fit(df).transform(df)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1995244107513451> in <module> 
 1 imputer = Imputer(
 2 inputCols = global_weather_table [ 'TEMP' ] , 
 ----> 3 outputCols = [ "{}_imputed" . format ( c ) for c in df [ 'TEMP' ] ] 
 4 ).setStrategy("median")
 5 

 TypeError : 'NoneType' object is not subscriptable

In [0]:
# Question 1
q1 = global_weather_table.select('COUNTRY_FULL', 'TEMP').groupby('COUNTRY_FULL').agg(avg(col("TEMP")).alias('AVG_TEMP')).orderBy(desc('AVG_TEMP')).limit(1)
q1.show()

+------------+-----------------+
COUNTRY_FULL| AVG_TEMP|
+------------+-----------------+
 DJIBOUTI|90.06114457831336|
+------------+-----------------+

In [0]:
q3 = global_weather_table.select('COUNTRY_FULL', 'WDSP').groupby('COUNTRY_FULL').agg(avg(col("WDSP")).alias('AVG_WIND_SPEED')).orderBy(desc('AVG_WIND_SPEED')).limit(1)
q3.show()

+------------+------------------+
COUNTRY_FULL| AVG_WIND_SPEED|
+------------+------------------+
 GABON|485.17947805453844|
+------------+------------------+

In [0]:
# Question 3
avg_df = global_weather_table.select('COUNTRY_FULL', 'WDSP').groupby('COUNTRY_FULL').agg(avg(col("WDSP")).alias('AVG_WIND_SPEED'))
windowSpec  = Window.partitionBy("COUNTRY_FULL").orderBy('AVG_WIND_SPEED')

ranked_df = avg_df.withColumn("rank", rank().over(windowSpec).alias('rank')) \
    .filter(col("rank") == 2).show()
#q3 = ranked_df.filter(col("rank") == 2)

+------------+--------------+----+
COUNTRY_FULL|AVG_WIND_SPEED|rank|
+------------+--------------+----+
+------------+--------------+----+